In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44034
2,Huelva,Confirmados PDIA 14 días,1734
3,Huelva,Tasa PDIA 14 días,"337,8997213399069"
4,Huelva,Confirmados PDIA 7 días,731
5,Huelva,Tasa PDIA 7 dias,"142,4479217413333"
6,Huelva,Total Confirmados,44232
7,Huelva,Curados,39329
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44034.0


/tmp/ipykernel_3517/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12251.0


/tmp/ipykernel_3517/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3517/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3517/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3517/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 882 personas en los últimos 7 días 

Un positivo PCR cada 321 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44034.0,731.0,1734.0,513170.0,142.447922,337.899721,240.0
Huelva-Costa,26009.0,477.0,1125.0,289548.0,164.739525,388.536616,162.0
Huelva (capital),12251.0,163.0,448.0,143837.0,113.322719,311.463671,60.0
Condado-Campiña,13723.0,187.0,441.0,156231.0,119.694555,282.274325,53.0
Cartaya,1828.0,62.0,129.0,20083.0,308.718817,642.334313,25.0
Sierra de Huelva-Andévalo Central,3891.0,56.0,148.0,67391.0,83.097149,219.613895,21.0
Lepe,2927.0,63.0,126.0,27880.0,225.968436,451.936872,18.0
Aljaraque,1597.0,25.0,50.0,21474.0,116.419857,232.839713,14.0
Isla Cristina,3006.0,60.0,143.0,21393.0,280.465573,668.442949,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Linares de la Sierra,11.0,2.0,7.0,262.0,763.358779,2671.755725,1.0
Almendro (El),57.0,13.0,18.0,840.0,1547.619048,2142.857143,8.0
Cañaveral de León,26.0,5.0,8.0,396.0,1262.626263,2020.202020,0.0
Alájar,34.0,3.0,12.0,759.0,395.256917,1581.027668,0.0
Rosal de la Frontera,67.0,10.0,21.0,1697.0,589.275192,1237.477902,6.0
Cabezas Rubias,51.0,7.0,7.0,706.0,991.501416,991.501416,3.0
Escacena del Campo,224.0,14.0,17.0,2287.0,612.155662,743.331876,1.0
Isla Cristina,3006.0,60.0,143.0,21393.0,280.465573,668.442949,13.0
Cartaya,1828.0,62.0,129.0,20083.0,308.718817,642.334313,25.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,192.0,1.0,10.0,3457.0,28.926815,289.268152,0.0,0.100000
Bollullos Par del Condado,1255.0,5.0,40.0,14387.0,34.753597,278.028776,-1.0,0.125000
Cumbres Mayores,129.0,1.0,8.0,1749.0,57.175529,457.404231,0.0,0.125000
Nerva,246.0,1.0,4.0,5169.0,19.346102,77.384407,0.0,0.250000
Alájar,34.0,3.0,12.0,759.0,395.256917,1581.027668,0.0,0.250000
Rociana del Condado,655.0,5.0,19.0,7939.0,62.980224,239.324852,1.0,0.263158
Linares de la Sierra,11.0,2.0,7.0,262.0,763.358779,2671.755725,1.0,0.285714
Cortegana,286.0,2.0,7.0,4602.0,43.459365,152.107779,2.0,0.285714
Palos de la Frontera,985.0,6.0,20.0,11742.0,51.098620,170.328734,1.0,0.300000
